In [4]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_1x_coco.py')

root='../dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_coco_trash'

#cfg.model.roi_head.bbox_head.num_classes = 10

# for cascade r-cnn
for bbox_head in cfg.model.roi_head.bbox_head:
    bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [6]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/detection/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


Use load_from_local loader


In [97]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[                                ] 2/4871, 1.8 task/s, elapsed: 1s, ETA:  2748s

/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 12.3 task/s, elapsed: 396s, ETA:     0s

In [98]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['image_id'] = file_names
submission['PredictionString'] = prediction_strings
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,image_id,PredictionString
0,test/0000.jpg,0 0.21594998 341.73782 636.7697 396.9904 726.9...
1,test/0001.jpg,0 0.7141323 760.80005 664.88135 896.1344 995.5...
2,test/0002.jpg,0 0.6894192 292.68713 327.41306 1014.92175 378...
3,test/0003.jpg,0 0.22688335 79.09911 88.105804 206.20255 198....
4,test/0004.jpg,0 0.3006426 425.70517 259.6507 687.42444 567.2...


## Visualization

In [99]:
# !apt install libcurl3 --yes
# !pip uninstall fiftyone-db-ubuntu1604 --yes
# !pip uninstall fiftyone --yes

In [1]:
import fiftyone as fo
from PIL import Image
from torchvision.transforms import functional as func

In [116]:
test_label = pd.read_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'),)

In [176]:
import json
annos_path = '../dataset/test.json'
with open(annos_path, 'rt', encoding='UTF-8') as annotations:
        coco = json.load(annotations)
        info = coco['info']
        licenses = coco['licenses']
        images = coco['images']
        annotations = coco['annotations']
        categories = coco['categories']

In [158]:
test_label = test_label[test_label['PredictionString'].isna()==False]

In [177]:
annotations

[]

In [184]:
test_label

,image_id,PredictionString
0,test/0000.jpg,0 0.21594998 341.73782 636.7697 396.9904 726.9...
1,test/0001.jpg,0 0.7141323 760.80005 664.88135 896.1344 995.5...
2,test/0002.jpg,0 0.6894192 292.68713 327.41306 1014.92175 378...
3,test/0003.jpg,0 0.22688335 79.09911 88.105804 206.20255 198....
4,test/0004.jpg,0 0.3006426 425.70517 259.6507 687.42444 567.2...
...,...,...
4866,test/4866.jpg,0 0.5861999 295.73572 340.97736 667.60187 686....
4867,test/4867.jpg,0 0.40928185 631.9767 664.2442 774.4623 735.39...
4868,test/4868.jpg,0 0.7795237 549.6743 542.9815 678.0216 772.374...
4869,test/4869.jpg,0 0.49549615 472.7379 408.40784 658.49805 481....


In [178]:
anno_id = 0
for idx,label in enumerate(test_label['PredictionString']):
    image_id = int(os.path.splitext(test_label['image_id'][idx].split('/'))[0])
    predict_list = label.split()
        
    for i in range(0,len(predict_list),6):
        anno={}
        
        anno['image_id'] = image_id
        anno['category_id'] = int(predict_list[i])
        x,y,width,height = float(predict_list[i+2]),float(predict_list[i+3]),float(predict_list[i+4]),float(predict_list[i+5])
        anno['bbox'] = [x,y,width,height]
        anno['segmentation'] = []
        anno['area'] = 0
        anno['iscrowd'] = 0
        anno['id'] = anno_id
        anno_id +=1
        
        annotations.append(anno) 

In [179]:
coco['annotations']

[{'image_id': 0,
  'category_id': 0,
  'bbox': [341.73782, 636.7697, 396.9904, 726.9023],
  'segmentation': [],
  'area': 0,
  'iscrowd': 0,
  'id': 0},
 {'image_id': 0,
  'category_id': 0,
  'bbox': [183.60303, 668.8007, 285.5107, 708.9399],
  'segmentation': [],
  'area': 0,
  'iscrowd': 0,
  'id': 1},
 {'image_id': 0,
  'category_id': 0,
  'bbox': [337.27502, 614.2204, 423.29852, 731.3875],
  'segmentation': [],
  'area': 0,
  'iscrowd': 0,
  'id': 2},
 {'image_id': 0,
  'category_id': 0,
  'bbox': [1.5164146, 295.43542, 61.21239, 546.0812],
  'segmentation': [],
  'area': 0,
  'iscrowd': 0,
  'id': 3},
 {'image_id': 0,
  'category_id': 0,
  'bbox': [0.985672, 455.47586, 77.27696, 543.4222],
  'segmentation': [],
  'area': 0,
  'iscrowd': 0,
  'id': 4},
 {'image_id': 0,
  'category_id': 0,
  'bbox': [190.31955, 675.78656, 296.24954, 735.0766],
  'segmentation': [],
  'area': 0,
  'iscrowd': 0,
  'id': 5},
 {'image_id': 0,
  'category_id': 0,
  'bbox': [331.54636, 564.0148, 462.7147,

In [180]:
coco['annotations'] = annotations
with open(os.path.join(cfg.work_dir, f'test_{epoch}.json'), 'wt', encoding='UTF-8') as f:
        json.dump(coco,f)

In [181]:
dataset = fo.Dataset.from_dir(dataset_type=fo.types.COCODetectionDataset,
                              data_path='../dataset',
                              labels_path=os.path.join(cfg.work_dir, f'test_{epoch}.json'))

 100% |███████████████| 4871/4871 [1.2m elapsed, 0s remaining, 71.4 samples/s]      


In [182]:
session = fo.launch_app(dataset)